<a href="https://colab.research.google.com/github/chahatpatel2003/CSCI-167/blob/main/notebook_12_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(3)
N = 3
D = 4
all_x = []
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
print(all_x)

np.random.seed(0)
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

all_queries = []
all_keys = []
all_values = []
for x in all_x:
  query = omega_q @ x + beta_q
  key = omega_k @ x + beta_k
  value = omega_v @ x + beta_v
  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

def softmax(items_in):
  a = np.array(items_in, dtype=float).reshape(-1,1)
  a = a - np.max(a)
  e = np.exp(a)
  out = (e / np.sum(e)).reshape(-1)
  return out

all_x_prime = []
for n in range(N):
  all_km_qn = []
  qn = all_queries[n]
  for key in all_keys:
    dot_product = float(key.T @ qn)
    all_km_qn.append(dot_product)
  attention = softmax(all_km_qn)
  print("Attentions for output ", n)
  print(attention)
  x_prime = np.zeros((D,1))
  for i, v in enumerate(all_values):
    x_prime += attention[i] * v
  all_x_prime.append(x_prime)

print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")

def softmax_cols(data_in):
  m = np.max(data_in, axis=0, keepdims=True)
  exp_values = np.exp(data_in - m)
  denom = np.sum(exp_values, axis=0, keepdims=True)
  softmax = exp_values / denom
  return softmax

def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
  Q = omega_q @ X + beta_q
  K = omega_k @ X + beta_k
  V = omega_v @ X + beta_v
  S = K.T @ Q
  A = softmax_cols(S)
  X_prime = V @ A
  return X_prime, A

X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

X_prime, A = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)
print(X_prime)
print("Attention matrix (unscaled):")
print(A)

def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
  Q = omega_q @ X + beta_q
  K = omega_k @ X + beta_k
  V = omega_v @ X + beta_v
  S = (K.T @ Q) / np.sqrt(X.shape[0])
  A = softmax_cols(S)
  X_prime = V @ A
  return X_prime, A

X_prime_scaled, A_scaled = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)
print(X_prime_scaled)
print("Attention matrix (scaled):")
print(A_scaled)

perm = np.array([2,0,1])
P = np.eye(N)[:,perm]
X_perm = X @ P
Xp1, _ = self_attention(X_perm,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)
Xp2, _ = scaled_dot_product_self_attention(X_perm,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)
print("Permutation covariance (unscaled):", np.allclose(Xp1, X_prime @ P, atol=1e-8))
print("Permutation covariance (scaled):", np.allclose(Xp2, X_prime_scaled @ P, atol=1e-8))


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]
Attentions for output  0
[1.24326146e-13 9.98281489e-01 1.71851130e-03]
Attentions for output  1
[2.79525306e-12 5.85506360e-03 9.94144936e-01]
Attentions for output  2
[0.00505708 0.00654776 0.98839516]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.0276

/tmp/ipython-input-1950077326.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dot_product = float(key.T @ qn)
